In [1]:
from astropy import units as u
from astropy.cosmology import LambdaCDM
from astropy.constants import L_sun
from vedo import Points, Axes, show
import matplotlib as mpl
mpl.use('pdf')

In [2]:
Data = pd.read_csv('results27_14_8_55_72.csv', index_col='specid')
Data = Data[(Data['bmag'] > 0)  & (Data['z_helio']>0)]

cosmo = LambdaCDM(H0=100, Om0=0.3, 
                  Ode0=0.7)
def D_C(z):
    return cosmo.comoving_distance(z).value

def D_L(z):
    return cosmo.luminosity_distance(z).value
Data['D C/Mpc'] = Data['z_helio'].map(D_C)
Data['D L/Mpc'] = Data['z_helio'].map(D_L)
Data.columns = [idx.replace('_', ' ') for idx in Data.columns]

def Mb(bmag, z):
    K = 2.6 * z + 4.3 * z ** 2
    mu = 5 * np.log10((cosmo.luminosity_distance(z).to(u.pc)/(10 * u.pc)).value)
    return bmag - mu - K
Data['Mb'] = list(map(Mb, *Data[['bmag', 'z helio']].values.T))
Data['L rel to sun'] = Data['Mb'].map(lambda Mb: 10 ** ((Mb- 5.31)/(-2.5)) * L_sun.to(u.erg/u.s).value)
def dVdz(z):
    return ((cosmo.differential_comoving_volume(z)* 17046 *u.deg * u.deg).to(u.mpc ** 3)/(1e36 * u.mpc ** 3)).value
Data.loc[:, 'dVdz'] = Data['z helio'].map(dVdz)


In [71]:
Data['L rel to sun'] = Data['Mb'].map(lambda Mb: 10 ** ((Mb- 5.31)/(-2.5)) * L_sun.to(u.erg/u.s).value)
Data['log L rel'] = Data['L rel to sun'].map(np.log10)

def dVdz(z):
    return ((cosmo.differential_comoving_volume(z)* 17046 *u.deg * u.deg).to(u.mpc ** 3)/(1e36 * u.mpc ** 3)).value
Data.loc[:, 'dVdz'] = Data['z helio'].map(dVdz)

data = Data[(Data['z helio'] < 0.11) & (Data['dVdz']>0)]

In [58]:
def mis(L, z):
    if 5.31 - 2.5 * np.log10(L/L_sun.to(u.erg/u.s).value) \
    + 5 * np.log10((cosmo.luminosity_distance(z).to(u.pc)/(10 * u.pc)).value) \
    + 2.6 * z + 4.3 * z ** 2 > 18:
        return 1
    else:
        return 0
data.loc[:, 'select'] = list(map(mis, *data[['L rel to sun', 'z helio']].values.T))

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [59]:
def L_lim(z):
    return np.log10((L_sun.to(u.erg /u.s).value) *10 ** ((5.31-18+ 5*np.log10((cosmo.luminosity_distance(z).to(u.pc)/(10 * u.pc)).value)+ (2.6 * z + 4.3 * z ** 2))/2.5))

In [60]:
Z = np.linspace(1e-10, 0.11, 100)
L_limit = [L_lim(z) for z in Z]

In [70]:
fig, ax = plt.subplots(1,1, figsize=(8,4))
ax.scatter(*data[['log L rel', 'z helio']].values.T,
           s=1, alpha=0.5, ec='none',color='C0')

ax.scatter(*data[data['select']==1][['log L rel', 'z helio']].values.T,
           s=1, alpha=0.5, ec='none',color='C2')
ax.plot(L_limit, Z, color='black')
ax.fill_betweenx(Z, L_limit, [38] * 100, ec='none',alpha=0, hatch='/')
ax.set_ylim([0, 0.11])
ax.set_xlim([38, 45])
ax.set_xlabel("$\log [L/(\mathrm{erg/s})]$")
ax.set_ylabel("$z$")
ax.annotate("$b_j>18$", (40, 0.06), bbox=dict(boxstyle="square", fc="w",ec='r'), fontsize=18)
plt.tight_layout()
fig.savefig("obs and not obs.jpeg",dpi=300)

In [72]:
from scipy.special import gamma
from scipy.integrate import nquad
def phi(L, alpha, L_star):
    return (L/L_star) ** alpha * np.exp(-L/L_star)
def p(L, dVdz, alpha, L_star):
    return phi(L, alpha, L_star) * dVdz

def x_max(z):
    return (-12.69 + 5 * np.log10((cosmo.luminosity_distance(z).to(u.pc)/(10 * u.pc)).value) \
    + 2.6 * z + 4.3 * z ** 2)/2.5

z_range = [0, 0.11]
def x_range(z):
    return [0, x_max(z)]
def int_term(alpha, L_star_rel):
    return (1/L_star_rel)**(alpha+1) * np.log(10)/gamma(alpha+1) * \
      nquad(lambda x, z: 10**((alpha+1) * x)*np.exp(-10**x /L_star_rel)* \
      ((cosmo.differential_comoving_volume(z)* 17046 *u.deg * u.deg).to(u.mpc ** 3)/(1e36 * u.mpc ** 3)).value,
                                                            [x_range, z_range])[0]
n = len(data)

data.loc[:, 'log dVdz'] = data['dVdz'].map(np.log10)
log10e = np.log10(np.e)
sum_log_V = sum(data['log dVdz'].values)
def P_obs(alpha, L_star):
    return -n * np.log10(L_star*gamma(alpha+1)) + sum_log_V +\
    np.sum( data['L rel to sun'].map(lambda l: alpha * np.log10(l/L_star) - log10e * l/L_star).values)
#np.sum(np.array(list(map(lambda L, dVdz: alpha * np.log10(L/L_star) - np.log10(np.e) * L/L_star + \
#                                  np.log10(dVdz), *data[['L rel to sun', 'dVdz']].values.T))))

def P(alpha, L_star, N):
    return 1/2 * np.log10(N) + N * (np.log(N)-1) - 1/2 * np.log10(N-n) + (N-n) * (np.log(N-n)-1) \
                + P_obs(alpha, L_star) + (N-n) * np.log10(int_term(alpha, L_star/3.828e+33))

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [79]:
alphas = np.linspace(-0.99, 0., 15)
L_stars = np.logspace(40, 45, 15)
res11 = np.zeros((15, 15))
for i in range(15):
    for j in range(15):
        res11[i, j] = P(alphas[i], L_stars[j],  10 * n)


In [80]:
np.save('res11.npy', res11)

In [81]:
(np.exp(res11/1e10+27)/4e2).max()

1329364583.0933824

In [82]:
(np.exp(res9/1e10+27)/4e2).max()

1329526289.3400588